In [156]:
import numpy as np
import pandas as pd
import os
from dataset import make_dataset
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [157]:
ss, not_scaled_df, scaled_df, data_train = make_dataset()
data_test = pd.read_csv('Data-test.csv', index_col=0)

In [158]:
# remove outliers from scaled and non_scaled dataframes
scaled_df = scaled_df.loc[scaled_df['sales'] <= 200].reset_index(drop=True)
not_scaled_df = not_scaled_df.loc[not_scaled_df['sales'] <= 200].reset_index(drop=True)

In [159]:
# creating the model
gbr = GradientBoostingRegressor(ccp_alpha=0, learning_rate=0.05, max_depth=3, min_samples_split=8, n_estimators=1500, random_state=24)

# # screating scaler and OHE object
ss = StandardScaler()
OHE = OneHotEncoder(sparse=False)

# adding interracting terms
ss, not_scaled_df, scaled_df, data_train = make_dataset()
not_scaled_df3 = not_scaled_df.copy()
not_scaled_df3 = not_scaled_df3[not_scaled_df3['sales'] <= 200]
not_scaled_df3 = not_scaled_df3.reset_index(drop=True)
not_scaled_df3['avgOriginalUnitPrice * avgFinalUnitPrice'] = not_scaled_df['avgOriginalUnitPrice'] * not_scaled_df['avgFinalUnitPrice']
not_scaled_df3['clickVolume*avgFinalUnitPrice'] = not_scaled_df['clickVolume'] * not_scaled_df['avgFinalUnitPrice']
not_scaled_df3['attribute1*avgFinalUnitPrice'] = not_scaled_df3['attribute1'] * not_scaled_df3['avgFinalUnitPrice']
not_scaled_df3['attribute2*avgFinalUnitPrice'] = not_scaled_df3['attribute2'] * not_scaled_df3['avgFinalUnitPrice']
not_scaled_df3['attribute1*avgOriginalUnitPrice'] = not_scaled_df3['attribute1'] * not_scaled_df3['avgOriginalUnitPrice']
not_scaled_df3['plus*meanPurchasePower'] = not_scaled_df3['plus'] * not_scaled_df3['meanPurchasePower']
not_scaled_df3['plus*meanUserLevel'] = not_scaled_df3['plus'] * not_scaled_df3['meanUserLevel']
not_scaled_df3['meanUserLevel*meanPurchasePower'] = not_scaled_df3['meanUserLevel'] * not_scaled_df3['meanPurchasePower']
y = not_scaled_df3.sales
not_scaled_df3 = not_scaled_df3.drop(['sales'], axis = 1)

# scaling
scaled_df3 = pd.DataFrame(ss.fit_transform(not_scaled_df3), columns = not_scaled_df3.columns)

# creating the test set
not_scaled_test = data_test.copy(deep=True)
categorical_list = ["productID", "brandID", "weekday"]
categorical_data = OHE.fit_transform(not_scaled_test[categorical_list])
categorical_df = pd.DataFrame(data=categorical_data, columns=OHE.get_feature_names_out())
not_scaled_test.drop(["productID", "brandID", "weekday"], axis=1, inplace=True)
not_scaled_test['avgOriginalUnitPrice * avgFinalUnitPrice'] = not_scaled_test['avgOriginalUnitPrice'] * not_scaled_test['avgFinalUnitPrice']
not_scaled_test['clickVolume*avgFinalUnitPrice'] = not_scaled_test['clickVolume'] * not_scaled_test['avgFinalUnitPrice']
not_scaled_test['attribute1*avgFinalUnitPrice'] = not_scaled_test['attribute1'] * not_scaled_test['avgFinalUnitPrice']
not_scaled_test['attribute2*avgFinalUnitPrice'] = not_scaled_test['attribute2'] * not_scaled_test['avgFinalUnitPrice']
not_scaled_test['attribute1*avgOriginalUnitPrice'] = not_scaled_test['attribute1'] * not_scaled_test['avgOriginalUnitPrice']
not_scaled_test['plus*meanPurchasePower'] = not_scaled_test['plus'] * not_scaled_test['meanPurchasePower']
not_scaled_test['plus*meanUserLevel'] = not_scaled_test['plus'] * not_scaled_test['meanUserLevel']
not_scaled_test['meanUserLevel*meanPurchasePower'] = not_scaled_test['meanUserLevel'] * not_scaled_test['meanPurchasePower']

# creating the scaled test set
not_scaled_test = pd.concat([categorical_df, not_scaled_test], axis=1)
test_set = pd.DataFrame(ss.transform(not_scaled_test), columns=not_scaled_test.columns)


In [160]:
# training the model
gbr.fit(scaled_df3, y)

GradientBoostingRegressor(ccp_alpha=0, learning_rate=0.05, min_samples_split=8,
                          n_estimators=1500, random_state=24)

In [161]:
# output the predicted sales
prediction_df = pd.DataFrame(gbr.predict(test_set).round(), columns=['Prediction'])
prediction_df.loc[prediction_df['Prediction'] < 0] = 0

In [162]:
Y_pred_GBR = prediction_df.Prediction
critical = 2/3

prediction_df['Inventory'] = (-Y_pred_GBR * np.log(1 -critical)).round()

In [ ]:
# current filepath
current_path = os.getcwd()
filename = current_path + 'Prediction.csv'
print(filename)

write_df = data_test.copy()
write_df['sales'] = prediction_df['Prediction']
write_df['inventory'] = prediction_df['Inventory']
# write to csv
write_df.to_csv('group04.csv')